In [38]:
from IO.config import get_usr_config
from IO.result import DetectionResult
from core import detector
from core import freq_transform
from core import signal_generator
from core import utils
from core.utils import MICROSECONDS_IN_SECONDS
from IO import config

import matplotlib.pyplot as plt
import numpy as np
import torch

from datetime import datetime

In [39]:
def unisign_quant(data, n_bits, clip, quant_flag=False):
    if not quant_flag:
        return data
    data = torch.Tensor(data)
    w_c = data.clamp(-clip, clip)
    b = torch.pow(torch.tensor(2.0), 1 - n_bits)

    w_q = clip * torch.min(b * torch.round(w_c / (b * clip)), 1 - b)

    return w_q.numpy()

def quant_routine(data, n_bits, clip, quant_flag=False):
    if data.dtype == 'complex128':
        real_q = unisign_quant(np.real(data), n_bits, np.real(clip), quant_flag=False)
        imag_q = unisign_quant(np.imag(data), n_bits, np.imag(clip), quant_flag)
        return real_q + 1j * imag_q
    else:
        return unisign_quant(data, n_bits, clip, quant_flag)
    
def get_min(data):
    if data.dtype == 'complex128':
        return np.min(np.abs(np.real(data))) + 1j*np.min(np.abs(np.imag(data)))
    else:
        return np.min(np.abs(data))
    
    
def get_max(data):
    if data.dtype == 'complex128':
        ret =  np.max(np.abs(np.real(data))) + 1j*np.max(np.abs(np.imag(data)))
    else:
        ret = np.max(np.abs(data))
    
    return ret
    
def dft_weight(N):
    return np.fft.fft(np.eye(N))/N

def sig_second_moment(sig):
    return (np.abs(sig)**2).mean()

def get_real_quantization_noise_var(delta):
    return delta**2/12

def get_complex_quantization_noise_var(delta):
    return delta**2/6

def get_formula_error(predicted, simulated):
    return 10*np.log10(predicted/simulated)

In [40]:
weight_quant = 0
Bw = 3
input_quant = 0
Bx = 4
acc_quant = 0
Ba = 8

In [24]:
noise_level = 0
configurations = config.parse_config('../yaml/example.yaml')

noise_level_db = noise_level
noise_level_linear = utils.db_to_linear(noise_level_db)

configurations.signal.num_pos_decision = 10000
configurations.signal.amps = [1]
configurations.signal.freqs = [406.25]
configurations.signal.phase = [0]
configurations.noise.init_args.top = noise_level_linear
configurations.noise.init_args.steady_state = noise_level_linear

generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
signal_with_noise, _ = generator.get()

configurations.noise.init_args.top = 0
configurations.noise.init_args.steady_state = 0

generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
signal_without_noise, _ = generator.get()

noise_sample = signal_with_noise - signal_without_noise

Nd = configurations.signal.num_pos_decision
N = configurations.signal.block_size
signal_with_noise = signal_with_noise[0:Nd,:,:]
signal_without_noise = signal_without_noise[0:Nd,:,:]
noise_sample = noise_sample[0:Nd,:,:]

In [25]:
coeff = freq_transform.dht_coeff_ditter(N)/N
wmax = get_max(coeff)

qcoeff = quant_routine(coeff, n_bits=Bw, clip=wmax, quant_flag=weight_quant)
deltaw = qcoeff - coeff
wsqnorm = np.real(np.array([np.conjugate(coeff[i,:]).dot(coeff[i,:]) for i in range(N)]))
deltawnorm = np.array([(np.conjugate(deltaw[i,:]).dot(deltaw[i,:])) for i in range(N)])

In [26]:
signal_power = np.var(signal_without_noise)
sigal_max = get_max(signal_without_noise)
qsignal_with_noise =  quant_routine(signal_with_noise, Bx, sigal_max, input_quant)
q_input_noise = qsignal_with_noise - signal_with_noise
deltax = utils.get_delta(Bx, sigal_max)
print(np.unique(qsignal_with_noise.reshape(N*Nd)).shape)
      

(160000,)


In [27]:
q_input_noise = q_input_noise.reshape(N*Nd)
q = q_input_noise[np.where(np.abs(q_input_noise) < deltax/2)[0]]
print(len(q)/N/Nd)


1.0


In [28]:
q.shape

(160000,)

In [29]:
result_noise = np.zeros(signal_without_noise.shape)
result_signal_without_noise = np.zeros(signal_without_noise.shape)
result_signal_with_noise = np.zeros(signal_without_noise.shape)
qresult_signal_without_noise = np.zeros(signal_without_noise.shape)
qresult_signal_with_noise = np.zeros(signal_without_noise.shape)

In [30]:
len(qnoise)/N/Nd

NameError: name 'qnoise' is not defined

In [35]:
start = datetime.now()
for i in range(Nd):
    result_signal_without_noise[i,:,:] = coeff.dot(signal_without_noise[0,:,:][0])
    result_noise[i,:,:]  = coeff.dot(noise_sample[i,:,:][0])
    result_signal_with_noise[i, : ,:] = coeff.dot(signal_with_noise[i,:,:][0])
    qresult_signal_with_noise[i, :, :] = qcoeff.dot(qsignal_with_noise[i,:,:][0])

result_max = get_max(result_signal_with_noise)
deltay = utils.get_delta(Ba, result_max)
qresult_signal_with_noise = quant_routine(qresult_signal_with_noise, Ba, result_max, acc_quant)
end = datetime.now()

final_noise = qresult_signal_with_noise - result_signal_without_noise

predicted_orig_noise_at_the_output = wsqnorm[3]*noise_level_linear
predicted_weight_quant_at_the_output = deltawnorm[3] * sig_second_moment(signal_with_noise) if weight_quant else 0
predicted_input_quant_at_the_output = wsqnorm[3] * q_input_noise.var() if input_quant else 0
predicted_acc_quant_at_the_output = get_real_quantization_noise_var(deltay) if acc_quant else 0

predicted_total = np.sum(np.array([
    predicted_orig_noise_at_the_output,
#     predicted_weight_quant_at_the_output,
#     predicted_input_quant_at_the_output,
#     predicted_acc_quant_at_the_output
])) 

simulated_noise_at_the_output = sig_second_moment(final_noise[:,:,3])


print('predicted vs. simulated:', 
      get_formula_error(predicted_total, simulated_noise_at_the_output), 'dB')

predicted vs. simulated: -8.779529407678854 dB


(10000, 1, 16)